In [1]:
# this chemkin file is from the cti generated by rmg

# load a bunch of stuff
from __future__ import division
# load
import cantera as ct
import numpy as np
import scipy
import pylab
import matplotlib
import matplotlib.pyplot  as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import NullFormatter, MaxNLocator, LogLocator
import csv
from pydas.dassl import DASSL
import os
%matplotlib inline

array = os.listdir('../cpox-cti')
files = ['../cpox-cti/' + x for x in array]

a = 68

gas = ct.Solution(files[a],'gas')
surf = ct.Interface(files[a],'surface1', [gas])
print "This mechanism contains %d gas reactions and %d surface reactions for the linear scaling file %s"%(gas.n_reactions, surf.n_reactions, array[a])
i_n2 = gas.species_index('N2')
i_ch4 = gas.species_index('CH4(2)')
i_o2 = gas.species_index('O2(3)')
i_co2 = gas.species_index('CO2(4)')
i_h2o = gas.species_index('H2O(5)')
i_h2 = gas.species_index('H2(6)')
i_co = gas.species_index('CO(7)')

This mechanism contains 64 gas reactions and 82 surface reactions for the linear scaling file 2.000o-2.750.cti


  1.000000e-04    0.100000    0.099997    0.800002
  2.238721e-04    0.100000    0.099997    0.800002
  5.011872e-04    0.100000    0.099997    0.800002
  1.122018e-03    0.100000    0.099997    0.800002
  2.511886e-03    0.100000    0.099997    0.800003
  5.623413e-03    0.099999    0.099997    0.800003
  1.258925e-02    0.099997    0.099996    0.800004
  2.818383e-02    0.099994    0.099994    0.800005
  6.309573e-02    0.099986    0.099991    0.800008
  1.412538e-01    0.099968    0.099982    0.800016
  3.162278e-01    0.099929    0.099962    0.800034
  7.079458e-01    0.099843    0.099916    0.800074
  1.584893e+00    0.099671    0.099816    0.800163
  3.548134e+00    0.099412    0.099642    0.800316
  7.943282e+00    0.099190    0.099470    0.800460
  1.778279e+01    0.099091    0.099379    0.800530
  3.981072e+01    0.099077    0.099352    0.800548
  8.912509e+01    0.099077    0.099343    0.800550
  1.995262e+02    0.099077    0.099327    0.800550
  4.466836e+02    0.099077    0

Text(0.5,0,u'Time(s)')

In [2]:
#######################################################################
# Input Parameters
#######################################################################

volume = 1.0
cat_area_per_vol = 1.0E2 #I made this up!

cat_area = cat_area_per_vol * volume

#initial conditions
T0 = 800.
P0 = 1.0 #bar
#initial composition (gas mol fraction)
CH4_0 = 0.1
O2_0 = 0.1
N2_0 = 0.8
CO2_0 = 0.

gas.TPX = T0, P0 * ct.one_atm, {'CH4(2)':CH4_0, 'O2(3)':O2_0, 'N2':N2_0, 'CO2(4)':CO2_0}
          #Temp, Pressure, mole fraction
surf.TP = T0, P0 * ct.one_atm
surf.coverages = {'X(1)':1.0}
#surf.coverages = {'PT(S)':0.9, 'O(S)':0.1}

r = ct.IdealGasConstPressureReactor(gas, energy='off')
r.volume = volume
rsurf = ct.ReactorSurface(surf, r, A=cat_area) # A=cat_area

sim = ct.ReactorNet([r])
sim.max_err_test_fails = 12

# set relative and absolute tolerances on the simulation
sim.rtol = 1.0e-12
sim.atol = 1.0e-18

#rxn_time = np.linspace(1E-12, 5, 10001) #
rxn_time = np.logspace(-4, 3, 10001) #from 0s to 100s, log spacing

gas_mole_fracs = np.zeros([gas.n_species, len(rxn_time)])
surf_site_fracs = np.zeros([surf.n_species, len(rxn_time)])
temperature = np.zeros(len(rxn_time))    
pressure = np.zeros(len(rxn_time))   

for i in range(len(rxn_time)):
    time = rxn_time[i] #define time in the reactor
    sim.advance(time) #Advance the simulation to next set time
    temperature[i] = gas.T
    pressure[i] = gas.P/ct.one_atm
    gas_mole_fracs[:,i] = gas.X #
    surf_site_fracs[:,i] = surf.coverages #
    if not i % 500:
        print('  {0:10e}  {1:10f}  {2:10f}  {3:10f}'.format(time, *gas['CH4(2)','O2(3)','N2'].X))



#sim.advance_to_steady_state()
#print('  {0:10e}  {1:10f}  {2:10f}  {3:10f}'.format(sim.time, *gas['CH4','H2','CO'].X))    
    
CH4_ref = gas_mole_fracs[i_ch4,:]

#Plot out simulations results
fig = pylab.figure(dpi=300,figsize=(8,8))
gs = gridspec.GridSpec(2, 1)
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1])

y_min = 1E-3

for i in range(gas.n_species):
    if i != i_n2:
        if np.max(gas_mole_fracs[i,:]) > y_min:
            ax0.loglog(rxn_time, gas_mole_fracs[i,:], label=gas.species_name(i) )

for i in range(surf.n_species):
    if np.max(surf_site_fracs[i,:]) > y_min:
        ax1.loglog(rxn_time, surf_site_fracs[i,:], label=surf.species_name(i) )
            
ax0.legend(loc='lower left', fontsize = 12)
ax1.legend(loc='lower left', fontsize = 12)

ax0.set_ylabel("gas-phase mole fraction")
ax0.set_ylim(y_min,0.5)
ax0.set_xlim(1.0E-4,max(rxn_time))
ax1.set_xlim(1.0E-4,max(rxn_time))
ax0.set_xlabel("Time(s)")
ax1.set_ylabel("surface site fraction")
#ax0.xaxis.set_major_locator(MaxNLocator(6))
#ax0.yaxis.set_major_locator(LogLocator(base=10.0, numticks=3))
#ax0.tick_params(axis='both', which='major', labelsize=10)
ax1.set_ylim(y_min, 1.1 )
ax1.set_xlabel("Time(s)")


In [3]:
# Set the value of the perturbation
dk = 1.0e-2
# create the array to hold the sensitivty at each time step
Sensitivity = np.zeros([surf.n_reactions,len(rxn_time)])

#cycle through each reaction
for m in range(surf.n_reactions):
    surf.set_multiplier(1.0) # reset all multipliers  
    surf.set_multiplier(1+dk, m) # perturb reaction m
    
    gas.TPX = T0, P0 * ct.one_atm, {'CH4(2)':CH4_0, 'O2(3)':O2_0, 'N2':N2_0}
              #Temp, Pressure, mole fraction
    surf.TP = T0, P0 * ct.one_atm
    surf.coverages = {'X(1)':1.0}
    
    r = ct.IdealGasConstPressureReactor(gas, energy='off')
    r.volume = volume
    rsurf = ct.ReactorSurface(surf, r, A=cat_area) # A=cat_area


    sim = ct.ReactorNet([r])
    sim.max_err_test_fails = 12


    # set relative and absolute tolerances on the simulation
    sim.rtol = 1.0e-12
    sim.atol = 1.0e-18
    gas_mole_fracs = np.zeros([gas.n_species, len(rxn_time)])
    temperature = np.zeros(len(rxn_time))    
    pressure = np.zeros(len(rxn_time))   

    for i in range(len(rxn_time)):
        time = rxn_time[i] #define time in the reactor
        sim.advance(time) #Advance the simulation to next set time
        temperature[i] = gas.T
        pressure[i] = gas.P/ct.one_atm
        gas_mole_fracs[:,i] = gas.X #[mol.frac,nth reactor]
    # here is the new output
    CH4_new = gas_mole_fracs[i_ch4,:]


    Sens = (CH4_new-CH4_ref)/(CH4_ref*dk)
    max_sensitivity = max(max(Sens),-min(Sens))
#    if (1==0):
#        if (max_sensitivity>0.01):
#            print "%d %s %.2F"%( m, surf.reaction_equations()[m],  max_sensitivity)
#    else:
    print "%d %s %.2F"%( m, surf.reaction_equations()[m],  max_sensitivity)
    Sensitivity[m,:] = Sens
    
surf.set_multiplier(1.0)

 

0 CH4(2) + X(1) <=> CH4X(33) 0.00
1 O2(3) + 2 X(1) <=> 2 OX(20) 0.00
2 C2H6(8) + 2 X(1) <=> 2 CH3X(22) 0.00
3 CH3(10) + X(1) <=> CH3X(22) 0.00
4 CH3X(22) + HOX(23) <=> CH4(2) + OX(20) + X(1) 0.01
5 2 HOX(23) <=> H2O(5) + OX(20) + X(1) 0.00
6 CH3OH(14) + 2 X(1) <=> CH3X(22) + HOX(23) 0.00
7 OH(17) + X(1) <=> HOX(23) 0.00
8 H2(6) + 2 X(1) <=> 2 HX(19) 0.00
9 CH3X(22) + HX(19) <=> CH4(2) + 2 X(1) 0.00
10 HOX(23) + HX(19) <=> H2O(5) + 2 X(1) 0.00
11 H(12) + X(1) <=> HX(19) 0.00
12 HOX(23) + X(1) <=> HX(19) + OX(20) 0.00
13 CH3OH(14) + X(1) <=> CH3OHX(46) 0.00
14 H2O(5) + X(1) <=> H2OX(35) 0.00
15 CH3OH(14) + 2 X(1) <=> CH3OX(44) + HX(19) 0.00
16 CH3OX(44) + X(1) <=> CH3X(22) + OX(20) 0.00
17 DME(138) + 2 X(1) <=> CH3OX(44) + CH3X(22) 0.00
18 DME(138) + X(1) <=> DMEX(142) 0.00
19 CH3OH(14) + 2 X(1) <=> CH3OX(45) + HX(19) 0.00
20 CH2OH(57) + X(1) <=> CH3OX(45) 0.00
21 HO2X(135) + X(1) <=> HOX(23) + OX(20) 0.00
22 HO2(52) + X(1) <=> HO2X(135) 0.00
23 CH2O(9) + X(1) <=> CH2OX(40) 0.00
24 OCO(1

Text(0,0.5,u'Sensitivity Coefficient')

In [7]:
#Plot out simulations results
fig = pylab.figure(dpi=300,figsize=(8,8))
gs = gridspec.GridSpec(1, 1)
ax0 = plt.subplot(gs[0])


for m in range(surf.n_reactions):
    y_max = 0.001
    if (1==1):
        if max(Sensitivity[m,:])>y_max or min(Sensitivity[m,:])<-y_max:
            lab = surf.reaction_equations()[m]
            ax0.semilogx(rxn_time, Sensitivity[m,:], label=lab )
    else:
        if max(Normed_sensitivity[m,:])>y_max or min(Normed_sensitivity[m,:])<-y_max:
            lab = surf.reaction_equations()[m]
            ax0.semilogx(rxn_time, Normed_sensitivity[m,:], label=lab )
        
ax0.legend(loc='best', fontsize = 12)   
ax0.set_xlim(rxn_time[1], rxn_time[-1] )

ax0.set_xlabel("Time(s)")
ax0.set_ylabel("Sensitivity Coefficient")

In [5]:
# now integrate with respect to time
from numpy import trapz

n = []
p = []
for m in range(surf.n_reactions):
    n.append(trapz(Sensitivity[m,:], rxn_time))#/max(rxn_time)
    p.append(surf.reaction_equations()[m])
#    if ans>1E-3:
#        print "%d %s %.2f"%( m, surf.reaction_equations()[m], ans )

import operator
ans = dict(zip(p,n))
sorted_ans = sorted(ans.items(), key=operator.itemgetter(1), reverse=True)

for x in sorted_ans:
    if x[1]>1E-3:
        print '%s %.3f'%(x)

CH3OH(14) + 2 X(1) <=> CH3OX(45) + HX(19) 2.940
HO2X(135) + X(1) <=> HOX(23) + OX(20) 1.174
CH2OH(57) + X(1) <=> CH3OX(45) 0.388
OCO(149) + 2 X(1) <=> HX(19) + SX(148) 0.280
OCO(149) + 2 X(1) <=> HX(19) + SX(160) 0.276
DME(138) + 2 X(1) <=> CH3OX(44) + CH3X(22) 0.201
DME(138) + 2 X(1) <=> HX(19) + SX(141) 0.071
CH3OH(14) + 2 X(1) <=> CH3OX(44) + HX(19) 0.016
CH3O(58) + X(1) <=> CH3OX(44) 0.016
SX(190) + X(1) <=> OX(20) + SX(160) 0.014
HOX(23) + HX(19) <=> H2O(5) + 2 X(1) 0.013
CH4O3(192) + 2 X(1) <=> HX(19) + SX(190) 0.006
2 HOX(23) <=> H2O(5) + OX(20) + X(1) 0.004
HO2(52) + X(1) <=> HO2X(135) 0.001
CH4O3(192) + 2 X(1) <=> HX(19) + SX(211) 0.001
